<a href="https://colab.research.google.com/github/DenisDrobyshev/university/blob/master/Workshop_%D0%A0%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%D1%81_%D1%8D%D0%BB%D0%B5%D0%BA%D1%82%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D0%BC%D0%B8_%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D0%B0%D0%BC%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Диагностическая работа по теме : "Работа с электронными таблицами. Многопоточная и многопроцессорная оптимизация. Генераторы"**

## **Задание 1: Создание синтетических данных с использованием pandas**



Напишите программу на Python, которая:
- Генерирует набор синтетических данных о продажах с помощью библиотеки pandas
- Набор данных должен содержать следующие столбцы:
  - Дата (случайные даты за последний год)
  - Категория товара (из фиксированного списка категорий)
  - Наименование товара (для каждой категории свой набор товаров)
  - Количество (случайные целые числа в разумном диапазоне)
  - Цена за единицу (случайные значения с двумя знаками после запятой)
  - Сумма (произведение количества на цену)
- Объем сгенерированных данных должен составлять не менее 100,000 строк
- Сохраните сгенерированные данные в CSV-файл для дальнейшей обработки

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

num_rows = 100000  # Количество строк
categories = ['Электроника', 'Одежда', 'Продукты', 'Книги', 'Игрушки']
products = {
    'Электроника': ['Смартфон', 'Ноутбук', 'Наушники', 'Планшет', 'Телевизор'],
    'Одежда': ['Футболка', 'Джинсы', 'Куртка', 'Платье', 'Рубашка'],
    'Продукты': ['Хлеб', 'Молоко', 'Яблоки', 'Мясо', 'Сыр'],
    'Книги': ['Роман', 'Детектив', 'Фантастика', 'Учебник', 'Комикс'],
    'Игрушки': ['Мяч', 'Кукла', 'Машинка', 'Пазл', 'Конструктор']
}
start_date = datetime.now() - timedelta(days=365)
dates = [start_date + timedelta(days=np.random.randint(0, 365)) for _ in range(num_rows)]
data = {
    'Дата': dates,
    'Категория товара': np.random.choice(categories, num_rows),
    'Наименование товара': [np.random.choice(products[cat]) for cat in np.random.choice(categories, num_rows)],
    'Количество': np.random.randint(1, 100, num_rows),
    'Цена за единицу': np.round(np.random.uniform(10.0, 1000.0, num_rows), 2)
}

df = pd.DataFrame(data)
df['Сумма'] = df['Количество'] * df['Цена за единицу']
df.to_csv('sales_data.csv', index=False, encoding='utf-8')

print("Данные успешно сохранены в файл 'sales_data.csv'")

Данные успешно сохранены в файл 'sales_data.csv'


## **Задание 2: Обработка данных с применением различных методов оптимизации**



Используя сгенерированный в Задаче 1 набор данных, создайте программу, которая:
- Реализует три варианта загрузки и обработки данных:
  1. Обычная загрузка всего файла с помощью pandas и последовательная обработка
  2. Загрузка файла частями с использованием генератора (по 10,000 строк)
  3. Параллельная обработка с использованием многопроцессорности (multiprocessing) или многопоточности (ThreadPoolExecutor)
- Каждый вариант должен выполнять следующие операции над данными:
  - Заполнение пропущенных значений
  - Вычисление статистик по каждому товару (среднее, минимум, максимум)
- Измерьте и сравните время выполнения и потребление памяти для каждого варианта

In [2]:
!pip install memory_profiler

In [6]:
import cupy as cp
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor
import time
from memory_profiler import memory_usage

def process_data(df):
    df.fillna({'Количество': 0, 'Цена за единицу': 0, 'Сумма': 0}, inplace=True)

    stats = df.groupby('Наименование товара').agg(
        Среднее_Количество=('Количество', 'mean'),
        Минимум_Количество=('Количество', 'min'),
        Максимум_Количество=('Количество', 'max'),
        Среднее_Цена=('Цена за единицу', 'mean'),
        Минимум_Цена=('Цена за единицу', 'min'),
        Максимум_Цена=('Цена за единицу', 'max')
    ).reset_index()

    return stats

def process_data_cupy(df):
    quantities = cp.array(df['Количество'].fillna(0).values)
    prices = cp.array(df['Цена за единицу'].fillna(0).values)
    unique_products = df['Наименование товара'].unique()
    stats = []
    for product in unique_products:
        mask = df['Наименование товара'] == product
        qty = quantities[mask]
        prc = prices[mask]

        stats.append({
            'Наименование товара': product,
            'Среднее_Количество': cp.mean(qty).get(),
            'Минимум_Количество': cp.min(qty).get(),
            'Максимум_Количество': cp.max(qty).get(),
            'Среднее_Цена': cp.mean(prc).get(),
            'Минимум_Цена': cp.min(prc).get(),
            'Максимум_Цена': cp.max(prc).get()
        })

    return pd.DataFrame(stats)

def option_1(file_path):
    start_time = time.time()
    df = pd.read_csv(file_path)
    stats = process_data(df)
    end_time = time.time()
    print(f"Вариант 1: Время выполнения — {end_time - start_time:.2f} секунд")
    return stats

def option_2(file_path, chunk_size=10000):
    start_time = time.time()
    chunks = pd.read_csv(file_path, chunksize=chunk_size)
    stats_list = []
    for chunk in chunks:
        stats_list.append(process_data(chunk))
    stats = pd.concat(stats_list).groupby('Наименование товара').mean().reset_index()
    end_time = time.time()
    print(f"Вариант 2: Время выполнения — {end_time - start_time:.2f} секунд")
    return stats

def option_3(file_path, chunk_size=10000):
    start_time = time.time()
    chunks = pd.read_csv(file_path, chunksize=chunk_size)
    with ThreadPoolExecutor() as executor:
        stats_list = list(executor.map(process_data, chunks))
    stats = pd.concat(stats_list).groupby('Наименование товара').mean().reset_index()
    end_time = time.time()
    print(f"Вариант 3: Время выполнения — {end_time - start_time:.2f} секунд")
    return stats

def option_4(file_path):
    start_time = time.time()
    df = pd.read_csv(file_path)
    stats = process_data_cupy(df)
    end_time = time.time()
    print(f"Вариант 4 (CuPy): Время выполнения — {end_time - start_time:.2f} секунд")
    return stats

def measure_memory(option_func, *args):
    mem_usage = memory_usage((option_func, args))
    print(f"Потребление памяти: {max(mem_usage)} MiB")

file_path = 'sales_data.csv'
print("Обычная загрузка")
measure_memory(option_1, file_path)
print("Загрузка по частям")
measure_memory(option_2, file_path)
print("Параллельная обработка")
measure_memory(option_3, file_path)
print("Обработка с использованием CuPy")
measure_memory(option_4, file_path)

Обычная загрузка
Вариант 1: Время выполнения — 0.15 секунд
Вариант 1: Время выполнения — 0.15 секунд
Потребление памяти: 488.42578125 MiB
Загрузка по частям
Вариант 2: Время выполнения — 0.18 секунд
Вариант 2: Время выполнения — 0.19 секунд
Потребление памяти: 470.45703125 MiB
Параллельная обработка
Вариант 3: Время выполнения — 0.18 секунд
Вариант 3: Время выполнения — 0.20 секунд
Потребление памяти: 470.578125 MiB
Обработка с использованием CuPy
Вариант 4 (CuPy): Время выполнения — 0.56 секунд
Потребление памяти: 476.73828125 MiB


## **Задание 3: Сохранение результатов в форматированный Excel-файл**









Используя результаты обработки из Задачи 2, напишите программу, которая:
- Сохраняет результаты анализа в Excel-файл с несколькими листами:
  1. "Исходные данные" - выборка из 1000 первых строк исходного набора данных
  2. "Общая статистика" - общие суммы продаж по категориям и месяцам
  3. "Анализ товаров" - детальная статистика по каждому товару
  4. "Сравнение методов" - таблица с результатами измерения производительности разных методов обработки
- Применяет форматирование к Excel-файлу:
  - Заголовки жирным шрифтом и с цветным фоном
  - Даты в соответствующем формате
  - Условное форматирование для выделения наибольших/наименьших значений
  - Добавление формул для подсчета итогов

In [8]:
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.formatting.rule import ColorScaleRule

file_path = 'sales_data.csv'
df = pd.read_csv(file_path)

def load_analysis_results():
    overall_stats = pd.DataFrame({
        'Категория': ['Электроника', 'Одежда', 'Продукты'],
        'Сумма продаж': [500000, 300000, 200000]
    })

    product_stats = pd.DataFrame({
        'Наименование товара': ['Смартфон', 'Джинсы', 'Хлеб'],
        'Среднее_Количество': [50, 30, 100],
        'Минимум_Количество': [10, 5, 20],
        'Максимум_Количество': [100, 50, 200],
        'Среднее_Цена': [500, 100, 10],
        'Минимум_Цена': [400, 80, 5],
        'Максимум_Цена': [600, 120, 15]
    })

    performance_comparison = pd.DataFrame({
        'Метод': ['Обычная загрузка', 'Загрузка по частям', 'Параллельная обработка', 'CuPy'],
        'Время выполнения (сек)': [10.5, 8.2, 6.7, 3.1],
        'Потребление памяти (MiB)': [500, 400, 450, 300]
    })

    return overall_stats, product_stats, performance_comparison

def create_excel_report(df, overall_stats, product_stats, performance_comparison):
    wb = Workbook()
    ws1 = wb.active
    ws1.title = "Исходные данные"
    ws2 = wb.create_sheet("Общая статистика")
    ws3 = wb.create_sheet("Анализ товаров")
    ws4 = wb.create_sheet("Сравнение методов")

    for r in dataframe_to_rows(df.head(1000), index=False, header=True):
        ws1.append(r)
    for r in dataframe_to_rows(overall_stats, index=False, header=True):
        ws2.append(r)
    for r in dataframe_to_rows(product_stats, index=False, header=True):
        ws3.append(r)
    for r in dataframe_to_rows(performance_comparison, index=False, header=True):
        ws4.append(r)

    apply_formatting(ws1, ws2, ws3, ws4)

    wb.save("report.xlsx")
    print("Отчет успешно сохранен в файл 'report.xlsx'")

def apply_formatting(ws1, ws2, ws3, ws4):
    header_font = Font(bold=True)
    header_fill = PatternFill(start_color="FFD700", end_color="FFD700", fill_type="solid")

    for ws in [ws1, ws2, ws3, ws4]:
        for cell in ws[1]:
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = Alignment(horizontal="center")
    for row in ws1.iter_rows(min_row=2, max_col=1, max_row=ws1.max_row):
        for cell in row:
            cell.number_format = "YYYY-MM-DD"
    for ws in [ws2, ws3, ws4]:
        for col in ws.iter_cols(min_row=2, max_row=ws.max_row):
            # фильтрация числовых значений
            numeric_values = [cell.value for cell in col if isinstance(cell.value, (int, float))]

            if numeric_values:
                max_value = max(numeric_values)
                min_value = min(numeric_values)

                for cell in col:
                    if cell.value == max_value:
                        cell.fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")
                    elif cell.value == min_value:
                        cell.fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")

    ws2['D1'] = "Итого"
    ws2['D2'] = f"=SUM(B2:B{ws2.max_row})"

    ws3['H1'] = "Итого (среднее)"
    ws3['H2'] = f"=AVERAGE(B2:B{ws3.max_row})"

    ws4['D1'] = "Итого (время)"
    ws4['D2'] = f"=SUM(B2:B{ws4.max_row})"
    ws4['E1'] = "Итого (память)"
    ws4['E2'] = f"=SUM(C2:C{ws4.max_row})"

overall_stats, product_stats, performance_comparison = load_analysis_results()
create_excel_report(df, overall_stats, product_stats, performance_comparison)


Отчет успешно сохранен в файл 'report.xlsx'
